In [ ]:
import sys
from copy import deepcopy
import time

sys.path.append('../utils')
from pyutils import *

In [ ]:
puzzle = readutf8('input.txt')
sample = readutf8('sample.txt')

In [ ]:
from PIL import Image, ImageDraw, ImageColor
def heatmap(mat: StrMatrix, distances: dict[Pt, int], dmin: int, dmax: int):
    im = Image.new(size=(len(mat[0]), len(mat)), mode='RGB', color='black')
    brush = ImageDraw.Draw(im)
    for pt, val in mat_iter(mat):
        if val != '#':
            brush.point((pt[1], pt[0]), 'white')
        if pt in distances:
            perc = (distances[pt] - dmin) / (dmax - dmin)
            hue = int(120 * perc)
            brush.point((pt[1], pt[0]), f'hsv({hue}, 100%, 100%)')
    return im

In [ ]:
def walk_maze(mat: StrMatrix, start_char: str, end_char: str):
    mat = deepcopy(mat)

    start_pt: Pt = Pt(-1, -1)
    end_pt: Pt = Pt(-1, -1)
    for pt, val in mat_iter(mat):
        if val == start_char:
            start_pt = Pt.of(pt)
        if val == end_char:
            end_pt = Pt.of(pt)
        if Pt(-1, -1) not in [start_pt, end_pt]:
            break

    assert(start_pt != Pt(-1, -1))
    assert(end_pt != Pt(-1, -1))

    visited: dict[Pt, int] = {}
    pos = start_pt
    dirpt: Pt = Pt(0, 1)
    score: int = 0

    log: list[str] = ['...']
    def _next_branch(pos: Pt):
        nonlocal dirpt, score
    
        # time.sleep(0.01)
        # clear_output(wait=True)
        # print(log[-1])
        # print(f'Now: {pos} ; {dirpt}')
        # print(f'Score: {score}')
        # display(heatmap(mat, visited, 0, end_pt.distance(start_pt)).resize((400, 400), resample=0))
        # print(show_in_matrix(mat, *visited, col='white on blue', colmap={pos:'black on red'}))

        visited[pos] = pos.distance(start_pt)
        for branch in Pt.cardinals():
            future = pos + branch
            log.append(f'Checking branch: {branch}')
            if future == end_pt:
                log.append(f'{future}: is END')
                return
            if future in visited:
                log.append(f'{future}: visited')
                continue
            if matget(mat, pos + branch) == '#':
                log.append(f'{future}: is a wall')
                continue
            if dirpt != branch:
                diff = abs(Pt.cardinals().index(branch) - Pt.cardinals().index(dirpt))
                if diff == 2:
                    score += 2000
                if diff in [1, 3]:
                    score += 1000
            dirpt = branch
            _next_branch(future)
            log.append(f'Out of branches')
    
    ret = _next_branch(pos)
    display(heatmap(mat, visited, 0, end_pt.distance(start_pt)).resize((600, 600), resample=0))
    return ret, visited

In [ ]:
maze = strtomat(sample)

In [ ]:
res = walk_maze(maze, 'S', 'E')